#### websocket으로 데이터 불러오기

https://technfin.tistory.com/entry/%ED%8C%8C%EC%9D%B4%EC%8D%AC-%EC%97%85%EB%B9%84%ED%8A%B8-%EC%9B%B9%EC%86%8C%EC%BC%93-%EC%A0%91%EC%86%8D%EB%B0%A9%EB%B2%95-%EB%B9%84%ED%8A%B8%EC%BD%94%EC%9D%B8-%EC%9E%90%EB%8F%99%EB%A7%A4%EB%A7%A4-%ED%94%84%EB%A1%9C%EA%B7%B8%EB%9E%A8

#### 급등주 찾기
https://technfin.tistory.com/entry/%EA%B8%89%EB%93%B1%EC%A3%BC-%EC%B0%BE%EA%B8%B0-%EC%97%85%EB%B9%84%ED%8A%B8-%ED%8C%8C%EC%9D%B4%EC%8D%AC-%EB%B9%84%ED%8A%B8%EC%BD%94%EC%9D%B8-%EC%9E%90%EB%8F%99%EB%A7%A4%EB%A7%A4-%ED%94%84%EB%A1%9C%EA%B7%B8%EB%9E%A8

In [1]:
import os
import sys
import time
import json
import datetime
import asyncio
import logging
import traceback
import websockets

In [2]:
os.getcwd()

'C:\\Users\\2020 LG 울트라북\\트머'

In [3]:
import psycopg2

In [4]:
# 실행 환경에 따른 공통 모듈 Import
sys.path.append(os.path.dirname(os.path.dirname('C:\\Users\\2020 LG 울트라북\\트머')))
import upbit
 
# 프로그램 정보
pgm_name = 'websocket'
pgm_name_kr = '업비트 Ticker 웹소켓'
 
 
# -----------------------------------------------------------------------------
# - Name : get_subscribe_items
# - Desc : 구독 대상 종목 조회
# -----------------------------------------------------------------------------
def get_subscribe_items():
    try:
        subscribe_items = []
 
        # KRW 마켓 전 종목 추출
        items = upbit.get_items('KRW', '')
 
        # 종목코드 배열로 변환
        for item in items:
            subscribe_items.append(item['market'])
 
        return subscribe_items
 
    # ---------------------------------------
    # Exception 처리
    # ----------------------------------------
    except Exception:
        raise
 


In [5]:
import os
import sys
import time
import json
import datetime
import asyncio
import logging
import traceback
import websockets
 
# 실행 환경에 따른 공통 모듈 Import
sys.path.append(os.path.dirname(os.path.dirname('C:\\Users\\2020 LG 울트라북\\트머')))
import upbit
 
# 프로그램 정보
pgm_name = 'websocket'
pgm_name_kr = '업비트 Ticker 웹소켓'
 
 
# -----------------------------------------------------------------------------
# - Name : get_subscribe_items
# - Desc : 구독 대상 종목 조회
# -----------------------------------------------------------------------------
def get_subscribe_items():
    try:
        subscribe_items = []
 
        # KRW 마켓 전 종목 추출
        items = upbit.get_items('KRW', '')
 
        # 종목코드 배열로 변환
        for item in items:
            subscribe_items.append(item['market'])
 
        return subscribe_items
 
    # ---------------------------------------
    # Exception 처리
    # ----------------------------------------
    except Exception:
        raise
 
 
# -----------------------------------------------------------------------------
# - Name : upbit_ws_client
# - Desc : 업비트 웹소켓
# -----------------------------------------------------------------------------
async def upbit_ws_client():
    try:
        # 중복 실행 방지용
        seconds = 0
 
        # 구독 데이터 조회
        subscribe_items = get_subscribe_items()
 
        logging.info('구독 종목 개수 : ' + str(len(subscribe_items)))
        logging.info('구독 종목 : ' + str(subscribe_items))
 
        # 구독 데이터 조립
        subscribe_fmt = [
            {"ticket": "test-websocket"},
            {
                "type": "ticker",
                "codes": subscribe_items,
                "isOnlyRealtime": True
            },
            {"format": "SIMPLE"}
        ]
 
        subscribe_data = json.dumps(subscribe_fmt)
 
        async with websockets.connect(upbit.ws_url) as websocket:
 
            await websocket.send(subscribe_data)
 
            while True:
                period = datetime.datetime.now()
 
                data = await websocket.recv()
                data = json.loads(data)
                logging.info(data)
 
                # 5초마다 종목 정보 재 조회 후 추가된 종목이 있으면 웹소켓 다시 시작
                if (period.second % 5) == 0 and seconds != period.second:
                    # 중복 실행 방지
                    seconds = period.second
 
                    # 종목 재조회
                    re_subscribe_items = get_subscribe_items()
                    logging.info('\n\n')
                    logging.info('*************************************************')
                    logging.info('기존 종목[' + str(len(subscribe_items)) + '] : ' + str(subscribe_items))
                    logging.info('종목 재조회[' + str(len(re_subscribe_items)) + '] : ' + str(re_subscribe_items))
                    logging.info('*************************************************')
                    logging.info('\n\n')
 
                    # 현재 종목과 다르면 웹소켓 다시 시작
                    if subscribe_items != re_subscribe_items:
                        logging.info('종목 달리짐! 웹소켓 다시 시작')
                        await websocket.close()
                        time.sleep(1)
                        await upbit_ws_client()
 
    # ----------------------------------------
    # 모든 함수의 공통 부분(Exception 처리)
    # ----------------------------------------
    except Exception as e:
        logging.error('Exception Raised!')
        logging.error(e)
        logging.error('Connect Again!')
 
        # 웹소켓 다시 시작
        await upbit_ws_client()

In [6]:
# -----------------------------------------------------------------------------
# - Name : main
# - Desc : 메인
# -----------------------------------------------------------------------------
async def main():
    try:
        # 웹소켓 시작
        await upbit_ws_client()
 
    except Exception as e:
        logging.error('Exception Raised!')
        logging.error(e)

In [7]:
# -----------------------------------------------------------------------------
# - Name : main
# - Desc : 메인
# -----------------------------------------------------------------------------
if __name__ == "__main__":
 
    # noinspection PyBroadException
    try:
        print("***** USAGE ******")
        print("[1] 로그레벨(D:DEBUG, E:ERROR, 그외:INFO)")
 
        if sys.platform.startswith('win32'):
            # 로그레벨(D:DEBUG, E:ERROR, 그외:INFO)
            log_level = 'I'
            upbit.set_loglevel(log_level)
        else:
            # 로그레벨(D:DEBUG, E:ERROR, 그외:INFO)
            log_level = sys.argv[1].upper()
            upbit.set_loglevel(log_level)
 
        if log_level == '':
            logging.error("입력값 오류!")
            sys.exit(-1)
 
        logging.info("***** INPUT ******")
        logging.info("[1] 로그레벨(D:DEBUG, E:ERROR, 그외:INFO):" + str(log_level))
 
        # ---------------------------------------------------------------------
        # Logic Start!
        # ---------------------------------------------------------------------
        # 웹소켓 시작
        async def my_coroutine():
            asyncio.run(main())
            

            if __name__ == '__main__':
                asyncio.run(my_coroutine())
        #asyncio.run(main())
 
    except KeyboardInterrupt:
        logging.error("KeyboardInterrupt Exception 발생!")
        logging.error(traceback.format_exc())
        sys.exit(-100)
 
    except Exception:
        logging.error("Exception 발생!")
        logging.error(traceback.format_exc())
        sys.exit(-200)

[2023/04/05 11:02:31 AM][INFO][1738909664.py:25]:***** INPUT ******
[2023/04/05 11:02:31 AM][INFO][1738909664.py:26]:[1] 로그레벨(D:DEBUG, E:ERROR, 그외:INFO):I


***** USAGE ******
[1] 로그레벨(D:DEBUG, E:ERROR, 그외:INFO)


In [8]:
# -----------------------------------------------------------------------------
# - Name : get_subscribe_items
# - Desc : 구독 대상 종목 조회
# -----------------------------------------------------------------------------
def get_subscribe_items():
    try:
        subscribe_items = []
 
        # KRW 마켓 전 종목 추출
        items = upbit.get_items('KRW', '')
 
        # 종목코드 배열로 변환
        for item in items:
            subscribe_items.append(item['market'])
 
        return subscribe_items
 
    # ---------------------------------------
    # Exception 처리
    # ----------------------------------------
    except Exception:
        raise

In [9]:
# -----------------------------------------------------------------------------
# - Name : upbit_ws_client
# - Desc : 업비트 웹소켓
# -----------------------------------------------------------------------------
async def upbit_ws_client():
    try:
        # 중복 실행 방지용
        seconds = 0
 
        # 구독 데이터 조회
        subscribe_items = get_subscribe_items()
 
        logging.info('구독 종목 개수 : ' + str(len(subscribe_items)))
        logging.info('구독 종목 : ' + str(subscribe_items))
 
        # 구독 데이터 조립
        subscribe_fmt = [
            {"ticket": "test-websocket"},
            {
                "type": "ticker",
                "codes": subscribe_items,
                "isOnlyRealtime": True
            },
            {"format": "SIMPLE"}
        ]
 
        subscribe_data = json.dumps(subscribe_fmt)
 
        async with websockets.connect(upbit.ws_url) as websocket:
 
            await websocket.send(subscribe_data)
 
            while True:
                period = datetime.datetime.now()
 
                data = await websocket.recv()
                data = json.loads(data)
                logging.info(data)
 
                # 5초마다 종목 정보 재 조회 후 추가된 종목이 있으면 웹소켓 다시 시작
                if (period.second % 5) == 0 and seconds != period.second:
                    # 중복 실행 방지
                    seconds = period.second
 
                    # 종목 재조회
                    re_subscribe_items = get_subscribe_items()
                    logging.info('\n\n')
                    logging.info('*************************************************')
                    logging.info('기존 종목[' + str(len(subscribe_items)) + '] : ' + str(subscribe_items))
                    logging.info('종목 재조회[' + str(len(re_subscribe_items)) + '] : ' + str(re_subscribe_items))
                    logging.info('*************************************************')
                    logging.info('\n\n')
 
                    # 현재 종목과 다르면 웹소켓 다시 시작
                    if subscribe_items != re_subscribe_items:
                        logging.info('종목 달리짐! 웹소켓 다시 시작')
                        await websocket.close()
                        time.sleep(1)
                        await upbit_ws_client()
 
    # ----------------------------------------
    # 모든 함수의 공통 부분(Exception 처리)
    # ----------------------------------------
    except Exception as e:
        logging.error('Exception Raised!')
        logging.error(e)
        logging.error('Connect Again!')
 
        # 웹소켓 다시 시작
        await upbit_ws_client()

## 가져와서 누적하는 작업 해야돼 

In [ ]:
# -----------------------------------------------------------------------------
# - Name : start_find_shoot
# - Desc : 급등주 찾기 로직
# -----------------------------------------------------------------------------
def start_find_shoot():
    try:
        
 
        data_cnt = 0
    
            # Connect to the PostgreSQL database
        conn = psycopg2.connect(host="203.252.101.69", 
                                database="upbit_db118a",
                                user="upbit_118a",
                                password="gksehd118a")
 
        # DB 연결
      #  conn = cx_Oracle.connect(upbit.get_env_keyvalue('d1k9z0GopOy7G4wUvYDu6P3ewkNOpjZaoCBMtXaN'), upbit.get_env_keyvalue('Qm6KXXP41ScLk6pb7UirdGWrTrz2XDWMizmWmxXA'), upbit.get_env_keyvalue('p_service'))
 
        # 커서 획득
        c = conn.cursor()
 
        """         
         WHERE A.PCNT_BY_ST > 10.0                      -- 시작가 현재 상승률 > 10.0%
           AND A.TRADE_CNT > 500                        -- 거래건수 > 500
         ORDER BY A.PCNT_BY_ST DESC                       -- 시작가 대비 현재 상승률 상위순으로 정렬
 
        """
 
        sql = "SELECT A.CODE \
              ,A.DT \
              ,A.ST_PRICE \
              ,A.EN_PRICE \
              ,A.MIN_PRICE \
              ,A.MAX_PRICE \
              ,A.TRADE_VOLUME \
              ,A.TRADE_CNT \
              ,A.PCNT_BY_ST \
              ,A.PCNT_BY_MAX \
              ,A.PCNT_BY_ST_MAX \
          FROM FIND_SHOOT_1MIN A \
         WHERE A.PCNT_BY_ST > 10.0 \
           AND A.TRADE_CNT > 500 \
         ORDER BY A.PCNT_BY_ST DESC"
 
        # ----------------------------------------------------------------------
        # 반복 수행
        # ----------------------------------------------------------------------
        while True:
 
            # SQL 수행
            c.execute(sql)
 
            """
            0  ,A.CODE \
            1  ,A.DT \
            2  ,A.ST_PRICE \
            3  ,A.EN_PRICE \
            4  ,A.MIN_PRICE \
            5  ,A.MAX_PRICE \
            6  ,A.TRADE_VOLUME \
            7  ,A.TRADE_CNT \
            8  ,A.PCNT_BY_ST \
            9  ,A.PCNT_BY_MAX \
            10 ,A.PCNT_BY_ST_MAX \    
            """
 
            rows = c.fetchall()
 
            # 대상조건 발견 시
            if len(rows) > 0:
 
                # 발견된 로우만큼 처리
                # 급등주 발견 텔레그렘 메세지 발송
                for row in rows:
                    logging.info(row)
 
                    message = '[급등주 발견]'
                    message = message + '\n\n종목코드:' + str(row[0])
                    message = message + '\n상승률:' + str(row[8])
                    message = message + '\n거래건수:' + str(row[7])
 
                    upbit.send_telegram_message(message)
 
                    logging.info('메세지 발송 완료!')
                    logging.info(message)
 
                # 중복 메세지 발송하지 않기 위해 60초간 Sleep
                time.sleep(60)
 
            # 조회 건수 체크
            data_cnt = data_cnt + 1
 
            if data_cnt % 1000 == 0:
                logging.info("Checking...[" + str(data_cnt) + "][" + str(len(rows)) + "]")
 
    # ---------------------------------------
    # 모든 함수의 공통 부분(Exception 처리)
    # ----------------------------------------
    except Exception:
        raise
 
 
# -----------------------------------------------------------------------------
# - Name : main
# - Desc : 메인
# -----------------------------------------------------------------------------
if __name__ == '__main__':
 
    # noinspection PyBroadException
    try:
 
        print("***** USAGE ******")
        print("[1] 로그레벨(D:DEBUG, E:ERROR, 그외:INFO)")
 
        # 로그레벨(D:DEBUG, E:ERROR, 그외:INFO)
        log_level = sys.argv[1].upper()
        upbit.set_loglevel(log_level)
 
        if log_level == '':
            logging.error("입력값 오류!")
            sys.exit(-1)
 
        logging.info("***** INPUT ******")
        logging.info("[1] 로그레벨(D:DEBUG, E:ERROR, 그외:INFO):" + str(log_level))
 
        # ---------------------------------------------------------------------
        # Logic Start!
        # ---------------------------------------------------------------------
        start_find_shoot()
 
    except KeyboardInterrupt:
        logging.error("KeyboardInterrupt Exception 발생!")
        logging.error(traceback.format_exc())
        sys.exit(-100)
 
    except Exception:
        logging.error("Exception 발생!")
        logging.error(traceback.format_exc())
        sys.exit(-200)